In [11]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV

In [9]:
data = pd.read_csv('Train.csv')

In [ ]:
data.head()

In [ ]:
data['os'] = data['client_agent'].str.split("/",n = 1, expand = True)[1].str.split(' ',n = 1, expand = True)[1]
data.os.value_counts()

In [ ]:
data['client'] = data['client_agent'].str.split("/",n = 1, expand = True)[0]
data['c_version'] = data['client_agent'].str.split("/",n = 1, expand = True)[1].str.split(' ',n = 1, expand = True)[0]
data.c_version.value_counts()
data.isna().sum()

In [10]:
# Create a label encoder object 
def MultiLabelEncoder(columnlist,dataframe):
    for i in columnlist:
        labelencoder_X=preprocessing.LabelEncoder()
        dataframe[i]=labelencoder_X.fit_transform(dataframe[i])
    return dataframe


def preprocess(data):
    data['client'] = data['client_agent'].str.split("/",n = 1, expand = True)[0]
    data["client"] = data["client"].astype("str")
    data['client'].fillna(data['client'].mode(), inplace=True)
    data['c_version'] = data['client_agent'].str.split("/",n = 1, expand = True)[1].str.split(' ',n = 1, expand = True)[0]
    data["c_version"] = data["c_version"].astype("str")
    data['c_version'].fillna(data['c_version'].mode(), inplace=True)
    data['device'] = data['device_details'].str.split("-",n = 1, expand = True)[0]
    data['browser'] = data['device_details'].str.split("-",n = 1, expand = True)[1]
    data.date = pd.to_datetime(data.date)
    data['year'] = data.date.dt.year
    data['month'] = data.date.dt.month
    data['day'] = data.date.dt.day
    data['dayofweek'] = data.date.dt.dayofweek
    data['month_start'] = data.date.dt.is_month_start
    data['month_end'] = data.date.dt.is_month_end
    cols = ['client','c_version','purchased','added_in_cart','checked_out','device','browser','month','day','dayofweek','month_start','month_end','time_spent']
    inn = data[cols]
    columnlist = ['client','device','browser','c_version','month_start','month_end']
    inn = MultiLabelEncoder(columnlist,inn)
    return inn

pre_data = preprocess(data)


/Users/subbuvidyasekar/Soft/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
pre_data.head()

In [4]:
scaler = MinMaxScaler(feature_range=(0.1, 0.9))
scaled = scaler.fit_transform(pre_data)
pre_data = pd.DataFrame(data = scaled, columns = pre_data.columns)

In [41]:
y = pre_data[['time_spent']]
x = pre_data.drop(['time_spent'], axis = 1)
x_value = np.array(x)
y_value = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(x_value, y_value,test_size=0.1,random_state=42)
gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,7),
            'n_estimators': (10, 50, 100, 1000),
            'min_samples_leaf':(2,4,6,8,10),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    
grid_result = gsc.fit(X_train, y_train)
best_params = grid_result.best_params_
best_params

/Users/subbuvidyasekar/Soft/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:715: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


In [42]:
best_params

{'max_depth': 6, 'min_samples_leaf': 2, 'n_estimators': 100}

In [75]:
def model():
    y = pre_data[['time_spent']]
    x = pre_data.drop(['time_spent'], axis = 1)
    x_value = np.array(x)
    y_value = np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(x_value, y_value,test_size=0.1,random_state=42)
    #lin_reg = LinearRegression()
    #lin_reg.fit(X_train, y_train)
    regr = RandomForestRegressor(n_estimators = 100, max_depth=12, 
                                 random_state=3, max_features= 'auto',
                                 min_samples_leaf= 6, min_samples_split= 10,
                                 bootstrap= True)
    regr.fit(X_train, y_train)
    predd = regr.predict(X_test)
    print(mean_squared_log_error(predd, y_test)) 
    return regr
lin_reg = model()

/Users/subbuvidyasekar/Soft/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]


3.1632892902156584


In [45]:
def preprocess_test(data):
    data['client'] = data['client_agent'].str.split("/",n = 1, expand = True)[0]
    data["client"] = data["client"].astype("str")
    data['client'].fillna(data['client'].mode(), inplace=True)
    data['c_version'] = data['client_agent'].str.split("/",n = 1, expand = True)[1].str.split(' ',n = 1, expand = True)[0]
    data["c_version"] = data["c_version"].astype("str")
    data['c_version'].fillna(data['c_version'].mode(), inplace=True)
    data['device'] = data['device_details'].str.split("-",n = 1, expand = True)[0]
    data['browser'] = data['device_details'].str.split("-",n = 1, expand = True)[1]
    data.date = pd.to_datetime(data.date)
    data['year'] = data.date.dt.year
    data['month'] = data.date.dt.month
    data['day'] = data.date.dt.day
    data['dayofweek'] = data.date.dt.dayofweek
    data['month_start'] = data.date.dt.is_month_start
    data['month_end'] = data.date.dt.is_month_end
    cols = ['client','c_version','purchased','added_in_cart','checked_out','device','browser','month','day','dayofweek','month_start','month_end']
    inn = data[cols]
    columnlist = ['device','browser','client','c_version','month_start','month_end']
    inn = MultiLabelEncoder(columnlist,inn)
    return inn

In [46]:
test = pd.read_csv('Test.csv')
sub = pd.read_csv("Sample Submission.csv")
pre_test = preprocess_test(test)
#scaler = MinMaxScaler(feature_range=(0.1, 0.9))
#scaled_test = scaler.fit_transform(pre_test)

/Users/subbuvidyasekar/Soft/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [47]:
pred = lin_reg.predict(pre_test)
pred

array([ 798.13320151,  208.44213237,  741.16403127, ...,  157.23068758,
        778.44800758, 1554.50445607])

In [48]:
sub['time_spent'] = pred
(sub < 0).values.any()

False

In [49]:
#sub['time_spent'] = sub['time_spent'].abs()
sub.to_csv("Submission.csv", index = False)

In [ ]:
type(pred)